In [1]:
import os
os.chdir('..')

In [2]:
import inspect
import multiprocessing as mp
from typing import List, Tuple

import numba as nb
import numpy as np
import pandas as pd
import time
from matplotlib import pyplot as plt

from neo_backtesting import Backtester, run_gridsearch
from strategy import boll_inbar

from util import read_candle_feather, transform_candle_np_struct, transform_np_struct
from joblib import Parallel, delayed

%matplotlib inline

In [3]:
# 回测起始日
START_DATE = '20180301'
END_DATE = '20220901'

# 策略参数
N = 100
B = 1.8
LONG_INTERVAL = '1h'

LEVERAGE = 1


# 回测参数
INIT_CAPITAL = 1e5  # 初始资金，10万
FACE_VALUE = 0.001  # 合约面值 0.001 币
FACE_VALUE_COIN = 10  # 合约面值 10 USD

COMM_RATE = 6e-4  # 交易成本万分之 6
LIQUI_RATE = 5e-3  # 爆仓保证金率千分之 5

CONTRACT_TYPE = 'inverse_futures'  # 正向合约

ETH_PATHS = {
    '1m': '/Users/lostleaf/feather_data/spot/ETH-USDT_1m.fea',
    '30m': '/Users/lostleaf/feather_data/spot/ETH-USDT_30m.fea',
    '1h': '/Users/lostleaf/feather_data/spot/ETH-USDT_1h.fea'
}

In [4]:
# 模拟器参数
simu_params = {
    'init_capital': INIT_CAPITAL, 
    'face_value': FACE_VALUE, 
    'comm_rate': 6e-4, 
    'liqui_rate': LIQUI_RATE, 
    'init_pos': 0
}

%time backtester = Backtester(ETH_PATHS, 'futures', simu_params, boll_inbar)

factor_params = {
    'n': N,
    'b': B,
    'itl': LONG_INTERVAL
}

strategy_params = {
    'leverage': LEVERAGE
}

%time df_usdt = backtester.run_detailed(START_DATE, END_DATE, INIT_CAPITAL, FACE_VALUE, \
                                        factor_params, strategy_params)
df_usdt['equity'] /= df_usdt['equity'].iat[0]
print(df_usdt[['candle_begin_time', 'close', 'pos', 'equity']])


CPU times: user 2.48 s, sys: 219 ms, total: 2.7 s
Wall time: 881 ms
CPU times: user 538 ms, sys: 170 ms, total: 708 ms
Wall time: 745 ms
          candle_begin_time    close       pos      equity
275636  2018-03-01 00:00:00   853.00         0    1.000000
275637  2018-03-01 00:01:00   852.80         0    1.000000
275638  2018-03-01 00:02:00   853.01         0    1.000000
275639  2018-03-01 00:03:00   852.97         0    1.000000
275640  2018-03-01 00:04:00   851.00         0    1.000000
...                     ...      ...       ...         ...
2638630 2022-08-31 23:56:00  1553.80  15620134  242.558917
2638631 2022-08-31 23:57:00  1554.28  15620134  242.633894
2638632 2022-08-31 23:58:00  1555.01  15620134  242.747921
2638633 2022-08-31 23:59:00  1554.10  15620134  242.605778
2638634 2022-09-01 00:00:00  1553.04  15620134  242.440204

[2362999 rows x 4 columns]


In [5]:
# 模拟器参数
init_capital = INIT_CAPITAL

simu_params = {
    'init_capital': init_capital, 
    'face_value': FACE_VALUE_COIN, 
    'comm_rate': 6e-4, 
    'liqui_rate': LIQUI_RATE, 
    'init_pos': 0
}

%time backtester = Backtester(ETH_PATHS, 'inverse_futures', simu_params, boll_inbar)

factor_params = {
    'n': N,
    'b': B,
    'itl': LONG_INTERVAL
}

strategy_params = {
    'leverage': LEVERAGE
}

%time df_coin = backtester.run_detailed(START_DATE, END_DATE, init_capital, FACE_VALUE_COIN, \
                                        factor_params, strategy_params)

df_coin['usd'] = df_coin['equity'] * df_coin['close']
df_coin['usd'] /= df_coin['usd'].iat[0]

print(df_coin[['candle_begin_time', 'close', 'pos', 'equity', 'usd']])

CPU times: user 508 ms, sys: 137 ms, total: 645 ms
Wall time: 487 ms
CPU times: user 447 ms, sys: 149 ms, total: 595 ms
Wall time: 614 ms
          candle_begin_time    close      pos        equity         usd
275636  2018-03-01 00:00:00   853.00        0  1.000000e+05    1.000000
275637  2018-03-01 00:01:00   852.80 -8530000  1.000232e+05    0.999998
275638  2018-03-01 00:02:00   853.01 -8530000  9.999862e+04    0.999998
275639  2018-03-01 00:03:00   852.97 -8530000  1.000033e+05    0.999998
275640  2018-03-01 00:04:00   851.00 -8530000  1.002348e+05    0.999998
...                     ...      ...      ...           ...         ...
2638630 2022-08-31 23:56:00  1553.80        0  1.329100e+07  242.104983
2638631 2022-08-31 23:57:00  1554.28        0  1.329100e+07  242.179774
2638632 2022-08-31 23:58:00  1555.01        0  1.329100e+07  242.293519
2638633 2022-08-31 23:59:00  1554.10        0  1.329100e+07  242.151728
2638634 2022-09-01 00:00:00  1553.04        0  1.329100e+07  241.98656

In [6]:
tmp = df_coin['usd'] / df_usdt['equity'] - 1
tmp.describe()

count    2.362999e+06
mean    -6.957448e-04
std      9.932150e-04
min     -2.382805e-03
25%     -1.887427e-03
50%     -2.681043e-04
75%      6.768313e-05
max      8.053049e-04
dtype: float64